In [ ]:
import requests

from config import cookies, headers

params = {
    'categoryIds': '205',
    'offset': '0',
    'filterParams': 'WyJ0b2xrby12LW5hbGljaGlpIiwiLTEyIiwiZGEiXQ==',
    'limit': '1000',
    'doTranslit': 'true',
    'context': 'v2dzaG9wX2lkZFMwMDJsY2F0ZWdvcnlfaWRzn2MyMDX/ZmNhdF9JZGMyMDX/',
}

response = requests.get('https://www.mvideo.ru/bff/products/v2/search', params=params, cookies=cookies, headers=headers).json()

In [124]:
products_ids = response['body']['products']

In [125]:
len(products_ids)

1000

In [126]:
products_ids

['30063534',
 '400410049',
 '400307033',
 '30074449',
 '400257859',
 '30070492',
 '400407204',
 '400268685',
 '30080789',
 '400365681',
 '30074463',
 '30074465',
 '30074464',
 '30070480',
 '30074323',
 '30074447',
 '30079343',
 '30063538',
 '30063201',
 '400307031',
 '30069420',
 '30077269',
 '30074324',
 '30077266',
 '30063535',
 '30074445',
 '400259081',
 '30074443',
 '30074328',
 '400285733',
 '30069422',
 '400257868',
 '400307028',
 '30070491',
 '30074325',
 '30064758',
 '30074444',
 '400423090',
 '30079347',
 '400378297',
 '30074326',
 '30069423',
 '30079345',
 '400327103',
 '30074329',
 '30070496',
 '400259093',
 '30074466',
 '30077268',
 '400307053',
 '30079344',
 '30070490',
 '400411465',
 '30074446',
 '30074421',
 '30070139',
 '30077721',
 '400376104',
 '400407205',
 '400404566',
 '30074327',
 '400282912',
 '400414015',
 '400378331',
 '30074420',
 '400394468',
 '400404211',
 '30070628',
 '30070624',
 '400352972',
 '400306204',
 '30075198',
 '400268686',
 '400335295',
 '4002830

In [155]:
import random
import time
import json
import requests

output_file = 'mv_items.json'
bad_ids_file = 'bad_ids.json'

class Item():
    def __init__(self):
        self.name: str
        self.brand: str
        self.all_properties: dict
        self.product_id: str

items = []
bad_ids = []
cnt = 0

for product_id in products_ids:
    cnt += 1
    print(f"🔍 {cnt}. Пробуем product_id: {product_id}")

    params_info = {
        'productId': product_id,
    }

    try:
        # Пауза между запросами
        time.sleep(random.uniform(1.5, 3.0))

        # Получаем информацию о товаре
        response = requests.get(
            'https://www.mvideo.ru/bff/product-details',
            params=params_info,
            cookies=cookies,
            headers=headers
        )

        if response.status_code != 200:
            print(f"⚠️ Пропущено product_id {product_id} — статус: {response.status_code}")
            bad_ids.append(product_id)
            continue

        response_info = response.json()

        new_item = Item()
        new_item.product_id = product_id
        new_item.name = response_info['body']['name']
        new_item.brand = response_info['body']['brandName']
        new_item.all_properties = response_info['body']['properties']['all']

        # Сохраняем сразу в файл построчно
        with open(output_file, "a", encoding="utf-8") as f:
            json.dump(new_item.__dict__, f, ensure_ascii=False)
            f.write("\n")

    except Exception as e:
        print(f"❌ Ошибка на product_id {product_id}: {e}")
        bad_ids.append(product_id)
        continue

# Сохраняем ID, которые не удалось обработать
with open(bad_ids_file, "w", encoding="utf-8") as f:
    json.dump(bad_ids, f, ensure_ascii=False)

print(f"\n✅ Готово. Успешно обработано: {cnt - len(bad_ids)}. Пропущено: {len(bad_ids)}.")


🔍 1. Пробуем product_id: 30063534
🔍 2. Пробуем product_id: 400410049
🔍 3. Пробуем product_id: 400307033
🔍 4. Пробуем product_id: 30074449
🔍 5. Пробуем product_id: 400257859
🔍 6. Пробуем product_id: 30070492
🔍 7. Пробуем product_id: 400407204
🔍 8. Пробуем product_id: 400268685
🔍 9. Пробуем product_id: 30080789
🔍 10. Пробуем product_id: 400365681
🔍 11. Пробуем product_id: 30074463
🔍 12. Пробуем product_id: 30074465
🔍 13. Пробуем product_id: 30074464
🔍 14. Пробуем product_id: 30070480
🔍 15. Пробуем product_id: 30074323
🔍 16. Пробуем product_id: 30074447
🔍 17. Пробуем product_id: 30079343
🔍 18. Пробуем product_id: 30063538
🔍 19. Пробуем product_id: 30063201
🔍 20. Пробуем product_id: 400307031
🔍 21. Пробуем product_id: 30069420
🔍 22. Пробуем product_id: 30077269
🔍 23. Пробуем product_id: 30074324
🔍 24. Пробуем product_id: 30077266
🔍 25. Пробуем product_id: 30063535
🔍 26. Пробуем product_id: 30074445
🔍 27. Пробуем product_id: 400259081
🔍 28. Пробуем product_id: 30074443
🔍 29. Пробуем product

In [148]:
import requests
import json
import time


def get_price_batch(batch):
    params_price = {
        'addBonusRubles': 'false',
        'isPromoApplied': 'true',
        'productIds': ','.join(batch)
    }
    response = requests.get(
        'https://www.mvideo.ru/bff/products/prices',
        params=params_price,
        cookies=cookies,
        headers=headers
    )
    return response.json()['body']['materialPrices']

all_price_dicts = []

# Обработка батчами по 100
for i in range(0, len(products_ids), 100):
    batch = products_ids[i:i + 100]
    try:
        prices = get_price_batch(batch)
        # Добавляем только price
        all_price_dicts.extend([item["price"] for item in prices if "price" in item])
        print(f"✅ Обработано: {len(all_price_dicts)} товаров")
        time.sleep(1.0)
    except Exception as e:
        print(f"❌ Ошибка на батче {i // 100 + 1}: {e}")

# Сохраняем в JSON
with open('only_prices.json', 'w', encoding='utf-8') as f:
    json.dump(all_price_dicts, f, ensure_ascii=False, indent=2)

print("🎉 Готово! Сохранено в only_prices.json")


✅ Обработано: 100 товаров
✅ Обработано: 200 товаров
✅ Обработано: 300 товаров
✅ Обработано: 400 товаров
✅ Обработано: 500 товаров
✅ Обработано: 600 товаров
✅ Обработано: 700 товаров
✅ Обработано: 800 товаров
✅ Обработано: 900 товаров
✅ Обработано: 1000 товаров
🎉 Готово! Сохранено в only_prices.json


In [147]:
# output_file = 

params_price = {
    'addBonusRubles' : 'false',
    'isPromoApplied' : 'true',
    'productIds' : ','.join(products_ids[:100])
    }


response_price = requests.get('https://www.mvideo.ru/bff/products/prices', params=params_price, cookies=cookies, headers=headers).json()
response_price['body']['materialPrices']
# new_item.all_prices = response_price['body']['materialPrices']
    
# with open(output_file, "a", encoding="utf-8") as f:
#         json.dump(new_item.__dict__, f, ensure_ascii=False)
#         f.write("\n")


[{'productId': '30074420',
  'price': {'productId': '30074420',
   'basePrice': 125999,
   'salePrice': 98999,
   'basePromoPrice': 98999,
   'discounts': [{'type': 'PRICE',
     'promoId': '7000056087',
     'promoName': 'Скидка по акции',
     'discount': 27000}],
   'applyPersonalPrice': False,
   'isFinalPrice': True}},
 {'productId': '400327084',
  'price': {'productId': '400327084',
   'basePrice': 17999,
   'salePrice': 12499,
   'basePromoPrice': 12499,
   'discounts': [{'type': 'PRICE',
     'promoId': '7000056087',
     'promoName': 'Скидка по акции',
     'discount': 5500}],
   'applyPersonalPrice': False,
   'isFinalPrice': False}},
 {'productId': '400257859',
  'price': {'productId': '400257859',
   'basePrice': 29999,
   'salePrice': 15999,
   'basePromoPrice': 15999,
   'discounts': [{'type': 'PRICE',
     'promoId': '7000056087',
     'promoName': 'Скидка по акции',
     'discount': 14000}],
   'applyPersonalPrice': False,
   'isFinalPrice': False}},
 {'productId': '300

In [122]:
params_info = {
'productId': '30063534',
}

params_price = {
'addBonusRubles' : 'false',
'isPromoApplied' : 'true',
'productIds' : '30063534'
}

response_info = requests.get('https://www.mvideo.ru/bff/product-details', params=params_info, cookies=cookies, headers=headers).json()
# print(response_info['body'])
new_item = Item()
new_item.name = response_info['body']['name']
new_item.brand = response_info['body']['brandName']
new_item.all_properties = response_info['body']['properties']['all']
response_price = requests.get('https://www.mvideo.ru/bff/products/prices', params=params_price, cookies=cookies, headers=headers)
response_price.content

b'<!DOCTYPE html>\n<html lang="ru">\n\n<head>\n  <title>\xd0\x93\xd0\xbb\xd0\xb0\xd0\xb2\xd0\xbd\xd0\xb0\xd1\x8f</title>\n  <meta charset="UTF-8">\n  <meta name="format-detection" content="telephone=no">\n  <link rel="stylesheet" href="https://static.mvideo.ru/media/Promotions/Promo_Page/2024/January/support/css/style.min.css">\n  <link rel="shortcut icon" href="favicon.ico">\n  <!-- <meta name="robots" content="noindex, nofollow"> -->\n  <!-- <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0"> -->\n  <!-- <meta name="viewport" content="width=device-width, user-scalable=no, initial-scale=1"> -->\n  <!-- <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"> -->\n  <meta name="viewport" content="width=device-width, initial-scale=1.0">\n</head>\n\n<!-- //static.mvideo.ru/media/Promotions/Promo_Page/2022/January/gibkoe-spisanie-bonusov/css/style.min.css -->\n\n<body>\n  <div class="wrapper">\n    <div c

In [ ]:
https://www.mvideo.ru/bff/products/prices?addBonusRubles=true&isPromoApplied=true&productIds=30063534

In [87]:
params = {
    'productId': '30063534',
}

price_params = {
    'addBonusRubles' : 'false',
    'isPromoApplied' : 'true',
    'productIds' : '30063534'
}

# response = requests.get('https://www.mvideo.ru/bff/product-details', params=params, cookies=cookies, headers=headers).json()
price_response = requests.get('https://www.mvideo.ru/bff/products/prices', params=price_params, cookies=cookies, headers=headers).json()

In [90]:
price_response['body']['materialPrices']

[{'productId': '30063534',
  'price': {'productId': '30063534',
   'basePrice': 74999,
   'salePrice': 49999,
   'basePromoPrice': 49999,
   'discounts': [{'type': 'PRICE',
     'promoId': '7000056087',
     'promoName': 'Скидка по акции',
     'discount': 25000}],
   'applyPersonalPrice': False,
   'isFinalPrice': True}}]

In [84]:
response['body']['properties']['all'][14]

{'name': 'Зарядка',
 'priority': 46,
 'properties': [{'id': 30852685,
   'name': 'Поддержка быстрой зарядки',
   'value': 'Да',
   'nameDescription': None,
   'valueDescription': None,
   'priority': 46,
   'measure': None,
   'iconPath': None},
  {'id': 30852803,
   'name': 'Поддержка беспроводной зарядки',
   'value': 'Да',
   'nameDescription': None,
   'valueDescription': None,
   'priority': 47,
   'measure': None,
   'iconPath': None},
  {'id': 624961,
   'name': 'Стандарты быстрой зарядки',
   'value': 'USB Power Delivery',
   'nameDescription': None,
   'valueDescription': None,
   'priority': 48,
   'measure': None,
   'iconPath': None}]}

In [38]:
response['body']['products']

[{'productId': '30063534',
  'name': 'Смартфон Apple iPhone 13 128GB nanoSim/eSim Midnight',
  'nameTranslit': 'smartfon-apple-iphone-13-128gb-nanosim-esim-midnight',
  'productType': 'DIG',
  'usedProductType': None,
  'materialCisNumber': '30063534',
  'materialSource': 'ERP',
  'modelName': 'iPhone 13 128GB Midnight',
  'rating': {'star': 4.915966386554622, 'count': 595, 'percent': 91},
  'supplier': None,
  'vendorCatalog': None,
  'sublicense': False,
  'saleLegal': True,
  'isMprimeSubscription': False,
  'isPreorder': False,
  'image': 'Pdb/small_pic/480/30063534b.jpg',
  'brandId': '685',
  'brandName': 'Apple',
  'status': {'showCase': False,
   'availableOnlyInRetailStore': False,
   'soldOut': False,
   'isPrepayment': True,
   'isCannotBeExchanged': False},
  'category': {'id': '914', 'name': 'iPhone'},
  'properties': None,
  'images': ['Pdb/30063534.jpg',
   'Pdb/30063534b.jpg',
   'Pdb/30063534b1.jpg',
   'Pdb/30063534b101.jpg',
   'Pdb/30063534b2.jpg',
   'Pdb/30063534b